In [1]:
#libraries
import json
from collections import defaultdict
import time

In [20]:
#import data from folder

def get_scene_graphs():
    with open('/nas/home/slnagark/Genome/datasets/scene_graphs.json') as f:
        scene_graphs = json.load(f)
    return scene_graphs

def get_image_data():
    with open('/nas/home/slnagark/Genome/datasets/image_data.json') as f:
        image_data = json.load(f)
    return image_data

def get_region_descriptions():
    with open('/nas/home/slnagark/Genome/datasets/region_descriptions.json') as f:
        region_descriptions = json.load(f)
    return region_descriptions

def get_objects():
    with open('/nas/home/slnagark/Genome/datasets/objects.json') as f:
        objects = json.load(f)
    return objects

def get_relationships():
    with open('/nas/home/slnagark/Genome/datasets/relationships.json') as f:
        relationships = json.load(f)
    return relationships

def get_region_graphs():
    with open('/nas/home/slnagark/Genome/datasets/region_graphs.json') as f:
        region_graphs = json.load(f)
    return region_graphs

def get_relationships():
    with open('/nas/home/slnagark/Genome/datasets/relationships.json') as f:
        relationships = json.load(f)
    return relationships

def get_relationship_synsets():
    with open('/nas/home/slnagark/Genome/datasets/relationship_synsets.json') as f:
        relationship_synets = json.load(f)
    return relationship_synets

def get_attribute_synsets():
    with open('/nas/home/slnagark/Genome/datasets/attribute_synsets.json') as f:
        attribute_synets = json.load(f)
    return attribute_synets

def get_object_synsets():
    with open('/nas/home/slnagark/Genome/datasets/object_synsets.json') as f:
        object_synets = json.load(f)
    return object_synets

def get_question_answers():
    with open('/nas/home/slnagark/Genome/datasets/question_answers.json') as f:
        q_a = json.load(f)
    return q_a

def get_mapping_qa_region():
    with open('/nas/home/slnagark/Genome/datasets/qa_to_region_mapping.json') as f:
        map_ = json.load(f)
    return map_

In [16]:
start = time.time()
map_ = get_mapping_qa_region()
print("Done importing Mapping QA to Regions in : ",time.time()-start," sec.")

Done importing Mapping QA to Regions in :  0.7821588516235352  sec.


In [3]:
start = time.time()
q_a_ = get_question_answers()
print("Done importing question and answers  in : ",time.time()-start," sec.")

Done importing question and answers  in :  5.874823808670044  sec.


In [4]:
start = time.time()
scene_graphs_ = get_scene_graphs()
print("Done importing scene_graphs in : ",time.time()-start," sec.")

Done importing scene_graphs in :  21.53645396232605  sec.


In [5]:
start = time.time()
region_graphs_ = get_region_graphs()
print("Done importing region graphs in : ",time.time()-start," sec.")

Done importing region graphs in :  74.083242893219  sec.


In [6]:
start = time.time()
relationships_ = get_relationships()
print("Done importing relationships in : ",time.time()-start," sec.")

Done importing relationships in :  27.39835834503174  sec.


In [89]:
#variables
relations_image_to_relations = {}
scene_graphs_image_to_relations = {}
rgraphs_image_to_relation = {}
relationid2name = defaultdict(list)
relationid2syns = defaultdict(list)
image_to_qa = {}
region_to_qa = defaultdict(list)

In [120]:
"""
input: relationship.json

output: list of dictionaries of relationships
        [
            {
                "rel_id" : int,
                "rel_name" : str,
                "rel_syns" : [str],
                "sub_id" : int,
                "sub_name" : str,
                "sub_syns" : [str],
                "ob_id" : int,
                "ob_name" : str,
                "ob_syns" : [str]
            },
            ....
        ]
"""

def get_info_from_relations(relationships):

    for ind, relations in enumerate(relationships):
        relations_id = []
 
        for rels in relations["relationships"]:
            rel_id = rels["relationship_id"]
            o_name = "name"
            s_name = "name"
            ob_ids = rels["object"]["object_id"]
            sub_ids = rels["subject"]["object_id"]
            if "merged_object_ids" in rels["object"].keys():
                o_name = "names"
                ob_ids = list(set(rels["object"]["merged_object_ids"]))
            if len(rels["subject"].keys())!=7:
                s_name = "names"
                sub_ids = list(set(rels["subject"]["merged_object_ids"]))
            relations_id.append({
                "rel_id" : rel_id,
                "rel_name" : rels["predicate"].lower(),
                "rel_syns" : rels["synsets"],
                "sub_id" : sub_ids,
                "sub_name" : rels["subject"][s_name],
                "sub_syns" : rels["subject"]["synsets"],
                "ob_id" : ob_ids,
                "ob_name" : rels["object"][o_name],
                "ob_syns" : rels["object"]["synsets"]
            })
            
        relations_image_to_relations[relations["image_id"]] = relations_id
        if ind == 9:
                break

In [121]:
get_info_from_relations(relationships_)

In [126]:
"""
input: scene_graphs.json

output: dictionary of list of dictionaries of relationships
        {
          image_id:  
          [
              {
                "rel_id" : int,
                "rel_name" : str,
                "rel_syns" : [str],
                "sub_id" : int,
                "sub_name" : str,
                "sub_syns" : [str],
                "ob_id" : int,
                "ob_name" : str,
                "ob_syns" : [str]
              },
              ....
          ]
        }
"""

def get_info_from_scene_graphs(scene_graphs):
    for ind, an_image in enumerate(scene_graphs):
        image_id = an_image['image_id']

        # OBJECTS
        objid2names=defaultdict(list)
        objid2syns=defaultdict(list)
        for o in an_image['objects']:
            obj_id = o['object_id']
            if obj_id not in objid2names.keys() or (objid2names[obj_id] and o["names"] not in objid2names[obj_id]): 
                objid2syns[obj_id] += o['synsets']
                objid2names[obj_id] += o["names"]

        # RELATIONS
        relations_id = []
        for rel in an_image['relationships']:
            rel_id = rel['relationship_id']
      
            if rel['predicate'].lower() not in relationid2name[rel_id]:
                relationid2name[rel_id] += rel['predicate'].lower()
                relationid2syns[rel_id] += rel["synsets"]

            sub_id=rel['subject_id']
            obj_id=rel['object_id']
            relations_id.append({
                        "rel_id" : rel_id,
                        "rel_name" : relationid2name[rel_id],
                        "rel_syns" : relationid2syns[rel_id],
                        "sub_id" : sub_id,
                        "sub_name" : objid2names[sub_id],
                        "sub_syns" : objid2syns[sub_id],
                        "ob_id" : obj_id,
                        "ob_name" : objid2names[obj_id],
                        "ob_syns" :objid2syns[obj_id]
                    })
            
        scene_graphs_image_to_relations[image_id] = relations_id
        if ind == 9:
            break

In [127]:
get_info_from_scene_graphs(scene_graphs_)

In [128]:
"""
input: region_graphs.json

output: dictionary of list of dictionaries of regions
        {
          image_id:  
          [
              {
                "region_id" : int,
                "phrase" : str,
                "relationship_id" : int,
              },
              ....
          ]
        }
"""
def get_info_from_region_graphs(region_graphs):

    for ind, reg_graphs in enumerate(region_graphs):
        region_data = []
        region_info = []
        image_id = reg_graphs["image_id"]
        
        for regions in reg_graphs['regions']:
            rel = []
            reg_info = {}
            
            r_objid2name = defaultdict(list)
            r_objid2syns = defaultdict(list)

            region_id = regions["region_id"]
            phrase = regions["phrase"]      

            reg_info["region_id"] = region_id
            reg_info["phrase"] = phrase

            
            for obj in regions["objects"]:
                ob_id = obj["object_id"]
                if ob_id not in r_objid2name.keys() or (r_objid2name[ob_id] and obj["name"] not in r_objid2name[ob_id]): 
                    r_objid2name[ob_id].append(obj["name"])
                    r_objid2syns[ob_id].append(obj["synsets"])
            
            
            for relations in regions["relationships"]:
                r_id = relations["relationship_id"]

                if r_id not in relationid2name.keys():
                    relationid2name[r_id].append(relations["predicate"].lower())
                    relationid2syns[r_id].append(relations["synsets"])
                    region_info.append({
                        "rel_id" : rels["relationship_id"],
                        "rel_name" : rels["predicate"],
                        "rel_syns" : rels["synsets"],
                        "sub_id" : rels["subject_id"],
                        "obj_id" : rels["object_id"],
                        "sub_name" : r_objid2name[rels["subject_id"]],
                        "obj_name" : r_objid2name[rels["object_id"]],
                        "sub_syns" : r_objid2syns[rels["subject_id"]],
                        "obj_syns" : r_objid2syns[rels["object_id"]]
                    })

                rel.append(r_id)

            reg_info["relationship_id"] = rel    
            region_data.append(reg_info)

        if len(region_info)!=0:
            print("Region info Activated")
            scene_graphs_image_to_relations[iamge_id].append(region_info)
    
        rgraphs_image_to_relation[image_id] = region_data
        if ind == 9:
            break

In [ ]:
get_info_from_region_graphs(region_graphs_)

In [131]:
"""
input: question_answers.json

output: dictionary of dictionary
        {
            "image_id" : {
                            "loc" : [{
                                        "question": str
                                        "answer"  : str
                                    }],
                                    
                            "time" : [{
                                        "question": str
                                        "answer"  : str
                                     }],
                                     
                            "event" :[{
                                        "question": str
                                        "answer"  : str
                                     }],
                          }
        }
"""
def get_info_from_q_a(q_a):

    for ind, qa in enumerate(q_a):
        loc = []
        time = []
        event = []
        cnt = 0
        image_id = qa["id"]
        for qas in qa["qas"]:
            qa_id = qas["qa_id"]
            question = qas["question"]
            answer = qas["answer"]
            q = question.split(" ")
            if q[0].lower() == "where":
                loc.append({
                    "question" : question,
                    "answer" : answer
                })
                
            elif q[0].lower() =="when":
                time.append({
                    "question" : question,
                    "answer" : answer
                })
              
            elif "doing" in question:
                event.append({
                    "question" : question,
                    "answer" : answer
                })

        image_to_qa[image_id] = {
            "location" : loc,
            "time" : time,
            "event" : event
        }
  
        if ind == 9:
            break


In [132]:
get_info_from_q_a(q_a_)